# Julia Fundamentals

## No types, no performance

Although types aren't always explicitly visible, **Julia is built around types**.

High performance codes in Julia make good use of the type system.

In [ ]:
3 + 2

In [ ]:
3 + 2.0

In [ ]:
typeof(3)

In [ ]:
typeof(2.0)

In [ ]:
typeof([1,2,3])

Here, `Array` is a **parametric type**. The type parameters are
* `Int64` (element type)
* `1` (dimensionality)

Hence `Array{Int64, 1}` means that we have a vector that can hold 64-bit integers.

In [ ]:
eltype([1,2,3])

In [ ]:
typeof(rand(3,3))

In [ ]:
typeof(5:7)

In [ ]:
3 isa Int64

In [ ]:
3 isa Float64

In [ ]:
3 isa Number

**Concrete types**
* types of values ("objects")
* specify data structure
* Example: `Float64`

**Abstract types**
* cannot be instantiated
* their descendant types share "behavior" ([duck typing](https://en.wikipedia.org/wiki/Duck_typing))
* Examples:

The abstract type `Number` indicates that one can do number-like things, e.g. `+`,`-`,`*`, and `/`, with corresponding values. In this category we have (concrete) things like `Float64` (double precision float) and `Int32` (single precision integer) numbers.

An `AbstractArray` is a type that, e.g., allows indexing `A[i]`. Examples include regular arrays (`Array`), as well as ranges (`UnitRange`).

### Inspecting the type tree

In [ ]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [ ]:
print_tree(Number) # print a branch of the type tree

Everything is a subtype of `Any`. It's the root of the type tree.

Note that **concrete types are the leaves of the type tree** whereas **abstract types are nodes** in the type graph.

In [ ]:
1 isa Integer

In [ ]:
Integer <: Number

In [ ]:
supertype(Integer)

### Duck typing example: `UnitRange`

**"If it walks like a duck and it quacks like a duck, then it must be a duck"**

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
typeof(x) <: AbstractArray

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all. In fact, **it's just two numbers**, the start and stop values of the range it represents.

In [ ]:
fieldnames(typeof(x))

It thus represents a range but with a much lower memory footprint than a corresponding array.

In [ ]:
sizeof([i for i in 1:10000000])

In [ ]:
sizeof(1:10000000)

 Note that allocating memory takes (a lot of) time and is a common performance bottleneck that one wants to avoid.

## Functions, methods, and dispatch

Let's define a *function* that calculates the absolute value of a number (like the built-in `abs` already does).

**How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?**

* Real number
  * "Drop the sign."
    * => `myabs(-4.32) = 4.32`
* Complex number:
  * "Square root of z times the complex conjugate of z."
    * => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [ ]:
myabs(x::Float64) = sign(x) * x

In [ ]:
myabs(-4.32)

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
methods(myabs)

One can check which particular method is being used through the `@which` macro.

In [ ]:
@which myabs(-4.32)

In [ ]:
@which myabs(1.0 + 1.0im)

Note that we should better loosen our type restrictions:

In [ ]:
myabs(-3)

In [ ]:
myabs(1 + 1im)

In [ ]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

In [ ]:
myabs(-3)

As we will understand later, type annotations in function signatures **virtually never affect performance**! One should therefore generally make them as generic as possible but as specific as necessary.

### Multiple dispatch

Which method gets executed when you call a generic function `f` for a given set of input arguments?

**Answer:** Julia always chooses the **most specific method** by considering **all input argument types**.

(Since methods belong to generic functions rather than objects no function argument is special.)

In [ ]:
f(a, b)                   = "a and b are anything"
f(a::Number, b)           = "a is a number, b is anything"
f(a, b::Number)           = "a is anything, b is a number"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::AbstractFloat, b::AbstractFloat) = "a and b are both floats"
f(a::Float32, b::Float32) = "a and b are both 32-bit floats"

In [ ]:
methods(f)

In [ ]:
f("Hello", "Julia")

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "Julia")

In [ ]:
f(1, 2)

In [ ]:
f(1.2, 3.4)

In [ ]:
f(1.2f0, 3.4f0) # 1.2f0 is the literal way to write Float32(1.2)

In [ ]:
@which f(1.2, 3.4)

In [ ]:
@which f(1, "UCL")

### Built-in Julia function

**Julia's built-in functions are not special by any means.**

In [ ]:
methods(+)

In [ ]:
@which true + false

In [ ]:
@which "Hello"*"World!"

We can easily modify or add methods to them as well.

In [ ]:
import Base: + # explicitly import functions to override/add methods

+(x::String, y::String) = x * " and " * y

In [ ]:
"A" + "B"

(**Type piracy**: as we neither *own* the `+` function nor the `String` type the above is bad practice and is only used for pedagogical reasons! 😉)

### Type parameters in function signatures (if time permits)

Naive approach:

In [ ]:
myfunc(v::Vector{Real}) = "I'm a real vector!"

In [ ]:
myfunc([1.0,2.0,3.0])

Huh? What's going on?

Note that although we have

In [ ]:
Float64 <: Real

parametric types have the following - at first somewhat counterintuitive - property

In [ ]:
Vector{Float64} <: Vector{Real}

In [ ]:
[1.0,2.0,3.0] isa Vector{Real}

The crucial point is that `Vector{Real}` describes a concrete **heterogeneous** vector: Each element can be of any type `T <: Real`.

In [ ]:
Real[1, 2.2, 13f0]

(Because it is a concrete type - a leaf of the type tree - it immediately follows that there are no subtypes.)

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [ ]:
myfunc(v::Vector{T}) where T<:Real = "I'm a real vector!"

In [ ]:
myfunc([1.0,2.0,3.0])

Here, `Vector{T} where T <: Real` describes all **homogeneous** vector types: Each vector can only hold elements of a specific type `T` that is a subtype of `Real`. This includes, for example: `Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ...`

(Side comment: We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way: `Vector{Real} === Vector{T where T<:Real}`. Note that the "degree of freedom", `T`, is inside of the curly braces.)

# Core messages of this notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* Types in function signatures serve as filters. **Avoid writing overly-specific types**.
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `Vector{T} where T<:SomeSuperType` to describe sets of restriced *homogeneous* vectors.